# Helmet Detection using CNN 

* 1= without Helmet 
* 0 = With Helmet

In [ ]:
pip install opencv-python

In [37]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image


from keras.models import Sequential
from keras.layers import Convolution2D  # convolution Layer
from keras.layers import MaxPooling2D   # Pooling Layer
from keras.layers import Flatten        # Flatten
from keras.layers import Dense   
        

In [38]:
# Initialising the CNN
classifier = Sequential()

############# CNN - 1 ##################

# Step 1 - Convolution Layer
classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu')) # calucaltion 

# Step 2 - Pooling Layer
classifier.add(MaxPooling2D(pool_size = (2, 2))) #  reduce the spatial size  two type of layer Avg & max layer



###  step 3 - Flattening Layer 

classifier.add(Flatten())        # conver  3D data to 1D data 

#################### Step 4 - Full connection  ( Neural Netwokr) ###################

classifier.add(Dense(units = 128, activation = 'relu'))   ## 1 HIDDEN LAYER WITH 128 NEURONS 
classifier.add(Dense(units = 1, activation = 'sigmoid'))   # relu use to convert -ve number into +ve number 

In [39]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [40]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 21, 21, 32)        896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 3200)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               409728    
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 410,753
Trainable params: 410,753
Non-trainable params: 0
________________________________________________

In [41]:
1.
# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)  # max normilization
                                   

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(r"C:\Data sciences all Important file\Bike riders wear a helment or not\Images\train",
                                                 target_size = (64, 64),  # HEIGHT AND WIDTH
                                                 batch_size = 32, # take 32 group which pick image randomly without replacement
                                                 class_mode = 'binary')  # binary because we have 2 type 

test_set = test_datagen.flow_from_directory(r"C:\Data sciences all Important file\Bike riders wear a helment or not\Images\test",
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 878 images belonging to 2 classes.
Found 448 images belonging to 2 classes.


In [42]:
training_set.class_indices

{'with_helmet': 0, 'without_helmet': 1}

In [43]:
import warnings 
warnings.filterwarnings("ignore")

In [44]:
result=classifier.fit(training_set,
                         epochs=20, # iteration or repeat 
                         validation_data = test_set)

Epoch 1/20
28/28 [==============================] - 51s 2s/step - loss: 0.7141 - accuracy: 0.5490 - val_loss: 0.6868 - val_accuracy: 0.5089
Epoch 2/20
28/28 [==============================] - 47s 2s/step - loss: 0.6656 - accuracy: 0.5854 - val_loss: 0.6635 - val_accuracy: 0.5625
Epoch 3/20
28/28 [==============================] - 55s 2s/step - loss: 0.6408 - accuracy: 0.6230 - val_loss: 0.6506 - val_accuracy: 0.5625
Epoch 4/20
28/28 [==============================] - 55s 2s/step - loss: 0.6128 - accuracy: 0.6515 - val_loss: 0.6542 - val_accuracy: 0.5647
Epoch 5/20
28/28 [==============================] - 52s 2s/step - loss: 0.5700 - accuracy: 0.6891 - val_loss: 0.5606 - val_accuracy: 0.7433
Epoch 6/20
28/28 [==============================] - 51s 2s/step - loss: 0.5404 - accuracy: 0.7301 - val_loss: 0.5370 - val_accuracy: 0.7567
Epoch 7/20
28/28 [==============================] - 50s 2s/step - loss: 0.5168 - accuracy: 0.7392 - val_loss: 0.5176 - val_accuracy: 0.7746
Epoch 8/20
28/28 [==

In [58]:
classifier.save('helmet_detect_classifier.h5')

In [59]:
from  keras.models import load_model

from tensorflow.keras.preprocessing import image

In [60]:
import matplotlib.pyplot as plt 
%matplotlib inline

In [61]:
import os 
os.chdir(r"C:\Data sciences all Important file\Bike riders wear a helment or not\Images\live_data")

In [62]:
model=load_model("helmet_detect_classifier.h5")

In [63]:
def model_output(path, model):
    raw_img = image.load_img(path, target_size=(64, 64))  # image is converted to 64*64
    
    raw_img = image.img_to_array(raw_img)    # convert image to array 
    
    raw_img = np.expand_dims(raw_img, axis=0)
    raw_img = raw_img / 255
    y_pred = model.predict(raw_img)
    
    predicted_class = int(y_pred[0][0] > 0.5)   # probability

    if predicted_class == 1:
        print("without_helmet")
        return "without_helmet"
    else:
        print("with_helmet")
        return "with_helmet"

In [64]:
def count_images_with_without_helmet(directory_path, model):
    
    with_helmet_count = 0
    without_helmet_count = 0

    for filename in os.listdir(directory_path):
        
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(directory_path, filename)
            prediction = model_output(image_path, model)
            
            if prediction == "without_helmet":
                without_helmet_count += 1
            else:
                with_helmet_count += 1

    return with_helmet_count, without_helmet_count





## Count Total number of bikers with and without Helmet

In [65]:
live_data_directory = r"C:\Data sciences all Important file\Bike riders wear a helment or not\Images\live_data"

with_helmet_count, without_helmet_count = count_images_with_without_helmet(live_data_directory, model)



1/1 [==============================] - 0s 158ms/step
with_helmet
1/1 [==============================] - 0s 52ms/step
with_helmet
1/1 [==============================] - 0s 47ms/step
without_helmet
1/1 [==============================] - 0s 47ms/step
with_helmet
1/1 [==============================] - 0s 55ms/step
without_helmet
1/1 [==============================] - 0s 54ms/step
without_helmet
1/1 [==============================] - 0s 46ms/step
with_helmet
1/1 [==============================] - 0s 63ms/step
with_helmet
1/1 [==============================] - 0s 64ms/step
with_helmet
1/1 [==============================] - 0s 62ms/step
without_helmet
1/1 [==============================] - 0s 63ms/step
with_helmet
1/1 [==============================] - 0s 56ms/step
with_helmet
1/1 [==============================] - 0s 62ms/step
with_helmet
1/1 [==============================] - 0s 42ms/step
with_helmet
1/1 [==============================] - 0s 63ms/step
without_helmet
1/1 [===================

In [66]:
print("Total number of bikers with helmet:", with_helmet_count)
print("Total number of bikers without helmet:", without_helmet_count)

Total number of bikers with helmet: 18
Total number of bikers without helmet: 9


## Result

### 1.Total number of bikers with helmet : 18
### 2.Total number of bikers without helmet : 9

In [67]:
image_path = "demo1.jpg"  
prediction_result = model_output(image_path, model)

1/1 [==============================] - 0s 47ms/step
with_helmet


In [68]:
 model_output("demo11.png",model)

1/1 [==============================] - 0s 74ms/step
without_helmet


'without_helmet'

## Detecte using VideoCapture

In [69]:
cap = cv2.VideoCapture(0)


In [70]:
while True:
    ret, frame = cap.read()
    
    img = cv2.resize(frame, (224, 224))
    
    ### call detect helmet method
    temp_img_path = 'temp_frame.jpg'  
    cv2.imwrite(temp_img_path, img)
    
    y_pred = model_output(temp_img_path,model)
          
    print(y_pred)

    if not ret:
        print("Failed to capture frame")
        break

    cv2.imshow("window", frame)

    # Check for user input every 1 millisecond
    key = cv2.waitKey(1) & 0xFF

    # Break the loop when the 'q' 
    if key == ord('q'):
        break

# Release the video capture and close OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 66ms/step
without_helmet
without_helmet
1/1 [==============================] - 0s 26ms/step
with_helmet
with_helmet
1/1 [==============================] - 0s 31ms/step
with_helmet
with_helmet
1/1 [==============================] - 0s 16ms/step
without_helmet
without_helmet
1/1 [==============================] - 0s 27ms/step
without_helmet
without_helmet
1/1 [==============================] - 0s 31ms/step
without_helmet
without_helmet
1/1 [==============================] - 0s 16ms/step
without_helmet
without_helmet
1/1 [==============================] - 0s 32ms/step
without_helmet
without_helmet
1/1 [==============================] - 0s 20ms/step
without_helmet
without_helmet
1/1 [==============================] - 0s 13ms/step
without_helmet
without_helmet
1/1 [==============================] - 0s 16ms/step
without_helmet
without_helmet
1/1 [==============================] - 0s 7ms/step
with_helmet
with_helmet
1/1 [==============================